In [1]:
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from torch.nn import CrossEntropyLoss


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from model.model import custom_resnet18

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4731, 0.4819, 0.4018], std=[0.1925, 0.1915, 0.1963])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4706, 0.4802, 0.4020], std=[0.1907, 0.1898, 0.1950])
    ]),
}

In [4]:
test_datasets = datasets.ImageFolder(
    './Seen Datasets/val',
    transform=data_transforms['val'],
)

In [5]:
batch_size = 64
test_dataloader = DataLoader(test_datasets, shuffle=False, batch_size=batch_size, num_workers=8, pin_memory=True)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.downsample = None
        if stride != 1 or in_planes != planes:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

class CustomResNet(nn.Module):
    def __init__(self, block, layers, num_classes=25):
        super(CustomResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(512 * block.expansion, num_classes)
        )

    def _make_layer(self, block, planes, blocks, stride=1):
        layers = []
        layers.append(block(self.in_planes, planes, stride))
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(block(planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def custom_resnet18(num_classes=25):
    return CustomResNet(BasicBlock, [2, 2, 2, 2], num_classes)


In [8]:
# model_with_state_dict = custom_resnet18().to(device)
model_with_state_dict = torch.load('best_bird_classifier_full.pth')
# model_with_state_dict.load_state_dict(state_dict)

In [9]:
def model_prediction(model, test_dataloader, device='cuda'):
    model.eval()
    targets = None
    predictions = None

    with torch.no_grad():
        for images, labels in tqdm(test_dataloader):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            results = model(images)
            if targets is None:
                targets = labels
            else:
                targets = torch.concat([targets, labels])
            
            if predictions is None:
                predictions = results
            else:
                predictions = torch.concat([predictions, results])

    return targets, predictions
        

In [10]:
def evaluation(model, test_loader, device='cuda'):
    st = time.time()
    targets, predictions = model_prediction(model, test_loader, device)
    et = time.time()
    elapsed_time = et - st
    pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Apply softmax to get probabilities
    probabilities = F.softmax(predictions, dim=1)

    # Convert to numpy arrays
    predictions_argmax_array = probabilities.argmax(1).cpu().numpy()
    predictions_array = probabilities.cpu().numpy()
    targets_array = targets.cpu().numpy()

    # Evaluation Metrics Computation
    accuracyScore = accuracy_score(targets_array, predictions_argmax_array)
    precisionScore = precision_score(targets_array, predictions_argmax_array, average='macro')
    recallScore = recall_score(targets_array, predictions_argmax_array, average='macro')
    f1Score = f1_score(targets_array, predictions_argmax_array, average='macro')
    roc_auc = roc_auc_score(targets_array, predictions_array, average='macro', multi_class='ovr')
    crossentropyloss = CrossEntropyLoss()(predictions, targets).item()

    result_dict = {
        'accuracy_score': accuracyScore,
        'precision_score': precisionScore,
        'recall_score': recallScore,
        'f1_score': f1Score,
        'roc_auc_score': roc_auc,
        'CrossEntropyLoss': crossentropyloss,
        'inference_time': elapsed_time,
        'model_parameters_count': pytorch_total_params
    }
    return result_dict

In [11]:
result = evaluation(model_with_state_dict, test_dataloader)
result

100%|██████████████████████████████████████████████████████| 118/118 [00:23<00:00,  5.03it/s]


{'accuracy_score': 0.9569333333333333,
 'precision_score': 0.9573784468447064,
 'recall_score': 0.9569333333333334,
 'f1_score': 0.9569104877550209,
 'roc_auc_score': 0.9991433518518518,
 'CrossEntropyLoss': 0.15825016796588898,
 'inference_time': 23.471702575683594,
 'model_parameters_count': 11189337}

In [12]:
team_name = str(input("Enter Team Name: "))

Enter Team Name:  AIIA


In [13]:
import json

with open(team_name + ".json", 'w') as file:
    json.dump(result, file)